In [1]:
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml import Pipeline
import pandas as pd
import numpy as np
import sklearn
import pandas as pd
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import Correlation
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [2]:
spark = SparkSession.builder.master('local[2]').config("spark.executor.memory", "1g").config("spark.driver.memory", "1g").appName('spark_sh_data').getOrCreate()

In [9]:
df = spark.read.options(header=None, nullValue='NULL', inferSchema=True).option('nullValue','null').csv('T4smallsubset.csv')

In [10]:
df.show()

+----------+-------------+---+-----+---------+---------+--------+---------+---------+--------+---------+--------+--------+----+----+----+----+----+----+----+----+----+----+----+----+--------------------+
|       _c0|          _c1|_c2|  _c3|      _c4|      _c5|     _c6|      _c7|      _c8|     _c9|     _c10|    _c11|    _c12|_c13|_c14|_c15|_c16|_c17|_c18|_c19|_c20|_c21|_c22|_c23|_c24|                _c25|
+----------+-------------+---+-----+---------+---------+--------+---------+---------+--------+---------+--------+--------+----+----+----+----+----+----+----+----+----+----+----+----+--------------------+
|0a50e09262|1451606401243|2.0|  2.4|     null|     null|    null|     null|     null|    null|     null|    null|    null|null|null|null|null|null|null|null|null|null|null|null|null|                null|
|0a50e09262|1451606691043|2.0|  2.4|      0.0|      0.0|     0.0|      0.0|      0.0| 8289872|134217728|29090442|20800570|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|20

In [11]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: long (nullable = true)
 |-- _c2: double (nullable = true)
 |-- _c3: double (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: integer (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: integer (nullable = true)
 |-- _c14: integer (nullable = true)
 |-- _c15: integer (nullable = true)
 |-- _c16: integer (nullable = true)
 |-- _c17: integer (nullable = true)
 |-- _c18: integer (nullable = true)
 |-- _c19: integer (nullable = true)
 |-- _c20: integer (nullable = true)
 |-- _c21: integer (nullable = true)
 |-- _c22: integer (nullable = true)
 |-- _c23: integer (nullable = true)
 |-- _c24: integer (nullable = true)
 |-- _c25: string (nullable = true)



In [12]:
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType
newDf = df.withColumn('_c4', translate('_c4', 'NULL', '')).withColumn('_c5', translate('_c5', 'NULL', '')).withColumn('_c6', translate('_c6', 'NULL', '')).withColumn('_c7', translate('_c7', 'NULL', ''))\
    .withColumn('_c8', translate('_c8', 'NULL', ''))
newDf1 = newDf.withColumn('_c4', newDf["_c4"].cast(IntegerType())).withColumn('_c5', newDf["_c5"].cast(IntegerType())).withColumn('_c6', newDf["_c6"].cast(IntegerType())).withColumn('_c7', newDf["_c7"].cast(IntegerType()))\
    .withColumn('_c8', newDf["_c8"].cast(IntegerType()))
newDf1.show()
newDf1.printSchema()

+----------+-------------+---+-----+----+----+----+----+----+--------+---------+--------+--------+----+----+----+----+----+----+----+----+----+----+----+----+--------------------+
|       _c0|          _c1|_c2|  _c3| _c4| _c5| _c6| _c7| _c8|     _c9|     _c10|    _c11|    _c12|_c13|_c14|_c15|_c16|_c17|_c18|_c19|_c20|_c21|_c22|_c23|_c24|                _c25|
+----------+-------------+---+-----+----+----+----+----+----+--------+---------+--------+--------+----+----+----+----+----+----+----+----+----+----+----+----+--------------------+
|0a50e09262|1451606401243|2.0|  2.4|null|null|null|null|null|    null|     null|    null|    null|null|null|null|null|null|null|null|null|null|null|null|null|                null|
|0a50e09262|1451606691043|2.0|  2.4|   0|   0|   0|   0|   0| 8289872|134217728|29090442|20800570|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|2016-01-01 02:04:...|
|0a50e09262|1451607965843|2.0|  2.4|  17|   0|   0|   0|   4| 3736234|134217728|34619480|30883246|  

In [15]:
from pyspark.sql.functions import rand,when

data= newDf1.withColumn('rand', when(rand() > 0.5, 'yes').otherwise('no'))

In [16]:
data.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: long (nullable = true)
 |-- _c2: double (nullable = true)
 |-- _c3: double (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: integer (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: integer (nullable = true)
 |-- _c14: integer (nullable = true)
 |-- _c15: integer (nullable = true)
 |-- _c16: integer (nullable = true)
 |-- _c17: integer (nullable = true)
 |-- _c18: integer (nullable = true)
 |-- _c19: integer (nullable = true)
 |-- _c20: integer (nullable = true)
 |-- _c21: integer (nullable = true)
 |-- _c22: integer (nullable = true)
 |-- _c23: integer (nullable = true)
 |-- _c24: integer (nullable = true)
 |-- _c25: string (nullable = true)
 |-- rand: string (nullable = false)



In [19]:
stringIndexer = StringIndexer().setInputCol("_c25").setOutputCol("_c25_index").setHandleInvalid("skip")
_c25_index_model=stringIndexer.fit(data)
_c25_index_df=_c25_index_model.transform(data)
_c25_index_df.toPandas().head()

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,...,_c18,_c19,_c20,_c21,_c22,_c23,_c24,_c25,rand,_c25_index
0,0a50e09262,1451606691043,2.0,2.4,0.0,0.0,0.0,0.0,0.0,8289872,...,0,0,0,0,0,0,0,2016-01-01 02:04:50.877,no,57.0
1,0a50e09262,1451607965843,2.0,2.4,17.0,0.0,0.0,0.0,4.0,3736234,...,0,0,0,0,0,0,0,2016-01-01 02:26:05.731,yes,312.0
2,0a50e09262,1451611415651,2.0,2.4,NaN,NaN,NaN,NaN,NaN,3460158,...,0,0,0,0,0,0,0,2016-01-01 03:23:35.539,no,1001.0
3,0a50e09262,1451608415819,2.0,2.4,NaN,NaN,NaN,NaN,NaN,1558612,...,1,129,2,78,1,129,2,2016-01-01 02:33:35.679,yes,402.0
4,0a50e09262,1451608030871,2.0,2.4,NaN,NaN,NaN,NaN,NaN,10359070,...,0,0,0,0,0,0,0,2016-01-01 02:27:10.762,no,325.0


In [20]:
encoder = OneHotEncoder().setInputCols(["_c25_index"]).setOutputCols(["_c25_encoded"])

encoder_model=encoder.fit(_c25_index_df)
encoder_df=encoder_model.transform(_c25_index_df)

encoder_df.toPandas().head()

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,...,_c19,_c20,_c21,_c22,_c23,_c24,_c25,rand,_c25_index,_c25_encoded
0,0a50e09262,1451606691043,2.0,2.4,0.0,0.0,0.0,0.0,0.0,8289872,...,0,0,0,0,0,0,2016-01-01 02:04:50.877,no,57.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,0a50e09262,1451607965843,2.0,2.4,17.0,0.0,0.0,0.0,4.0,3736234,...,0,0,0,0,0,0,2016-01-01 02:26:05.731,yes,312.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,0a50e09262,1451611415651,2.0,2.4,NaN,NaN,NaN,NaN,NaN,3460158,...,0,0,0,0,0,0,2016-01-01 03:23:35.539,no,1001.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,0a50e09262,1451608415819,2.0,2.4,NaN,NaN,NaN,NaN,NaN,1558612,...,129,2,78,1,129,2,2016-01-01 02:33:35.679,yes,402.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,0a50e09262,1451608030871,2.0,2.4,NaN,NaN,NaN,NaN,NaN,10359070,...,0,0,0,0,0,0,2016-01-01 02:27:10.762,no,325.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [26]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler().setInputCols(['_c4','_c5','_c6','_c7','_c8','_c9','_c10','_c11','_c12','_c13','_c14','_c15','_c16','_c17','_c18','_c19','_c20','_c21','_c22','_c23','_c24'])\
                    .setOutputCol("vectorized_features")\
                        .setHandleInvalid("skip")
                                  
assembler_df = assembler.transform(encoder_df)
assembler_df.toPandas().head()

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,...,_c20,_c21,_c22,_c23,_c24,_c25,rand,_c25_index,_c25_encoded,vectorized_features
0,0a50e09262,1451606691043,2.0,2.4,0,0,0,0,0,8289872,...,0,0,0,0,0,2016-01-01 02:04:50.877,no,57.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 8289872.0, 134217728.0, 29090442.0, 20800570.0, 0.0, 0.0, 0.0, 0.0, 0...."
1,0a50e09262,1451607965843,2.0,2.4,17,0,0,0,4,3736234,...,0,0,0,0,0,2016-01-01 02:26:05.731,yes,312.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(17.0, 0.0, 0.0, 0.0, 4.0, 3736234.0, 134217728.0, 34619480.0, 30883246.0, 0.0, 0.0, 0.0, 0.0, 0..."
2,0a50e09262,1451609770986,2.0,1.4,35,0,0,0,8,32466,...,0,0,0,0,0,2016-01-01 02:56:10.88,yes,672.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(35.0, 0.0, 0.0, 0.0, 8.0, 32466.0, 134217728.0, 31711276.0, 31678810.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
3,0a50e09262,1451608601266,2.0,0.3,13,0,0,0,3,9853730,...,0,0,0,0,0,2016-01-01 02:36:41.155,yes,439.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(13.0, 0.0, 0.0, 0.0, 3.0, 9853730.0, 134217728.0, 34230824.0, 24377094.0, 0.0, 0.0, 0.0, 0.0, 0..."
4,0a50e09262,1451606446297,2.0,0.3,11,0,0,0,2,10645694,...,0,0,0,0,0,2016-01-01 02:00:46.161,no,8.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(11.0, 0.0, 0.0, 0.0, 2.0, 10645694.0, 134217728.0, 31261076.0, 20615382.0, 0.0, 0.0, 0.0, 0.0, ..."


In [27]:
label_indexer = StringIndexer().setInputCol("rand").setOutputCol("label")

label_indexer_model=label_indexer.fit(assembler_df)
label_indexer_df = label_indexer_model.transform(assembler_df)

label_indexer_df.select("rand","label").toPandas().head()

,rand,label
0,no,1.0
1,yes,0.0
2,no,1.0
3,yes,0.0
4,no,1.0


In [28]:
scaler = StandardScaler().setInputCol("vectorized_features").setOutputCol("features")
scaler_model=scaler.fit(label_indexer_df)
scaler_df = scaler_model.transform(label_indexer_df)
pd.set_option('display.max_colwidth', 100)
scaler_df.select("vectorized_features","features").toPandas().head()

,vectorized_features,features
0,"(0.0, 0.0, 0.0, 0.0, 0.0, 8289872.0, 134217728.0, 29090442.0, 20800570.0, 0.0, 0.0, 0.0, 0.0, 0....","(0.0, 0.0, 0.0, 0.0, 0.0, 2.368964603686052, 0.0, 1.3712824528073548, 0.9540789824449748, 0.0, 0..."
1,"(17.0, 0.0, 0.0, 0.0, 4.0, 3736234.0, 134217728.0, 34619480.0, 30883246.0, 0.0, 0.0, 0.0, 0.0, 0...","(0.9473679756432686, 0.0, 0.0, 0.0, 0.3336051891155314, 1.067689114752116, 0.0, 1.63191351473157..."
2,"(35.0, 0.0, 0.0, 0.0, 8.0, 32466.0, 134217728.0, 31711276.0, 31678810.0, 0.0, 0.0, 0.0, 0.0, 0.0...","(1.9504634792655529, 0.0, 0.0, 0.0, 0.6672103782310628, 0.0092776830357901, 0.0, 1.4948248752951..."
3,"(13.0, 0.0, 0.0, 0.0, 3.0, 9853730.0, 134217728.0, 34230824.0, 24377094.0, 0.0, 0.0, 0.0, 0.0, 0...","(0.7244578637272053, 0.0, 0.0, 0.0, 0.25020389183664854, 2.815862245433869, 0.0, 1.6135928184362..."
4,"(11.0, 0.0, 0.0, 0.0, 2.0, 10645694.0, 134217728.0, 31261076.0, 20615382.0, 0.0, 0.0, 0.0, 0.0, ...","(0.6130028077691738, 0.0, 0.0, 0.0, 0.1668025945577657, 3.0421787293788105, 0.0, 1.4736030815440..."


In [29]:
pipeline_stages=Pipeline().setStages([stringIndexer,encoder,assembler,label_indexer,scaler])

pipeline_model=pipeline_stages.fit(data)
pipeline_df=pipeline_model.transform(data)

In [30]:
pipeline_df.toPandas().head()

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,...,_c22,_c23,_c24,_c25,rand,_c25_index,_c25_encoded,vectorized_features,label,features
0,0a50e09262,1451606691043,2.0,2.4,0,0,0,0,0,8289872,...,0,0,0,2016-01-01 02:04:50.877,no,57.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 8289872.0, 134217728.0, 29090442.0, 20800570.0, 0.0, 0.0, 0.0, 0.0, 0....",1.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 2.368964603686052, 0.0, 1.3712824528073548, 0.9540789824449748, 0.0, 0..."
1,0a50e09262,1451607965843,2.0,2.4,17,0,0,0,4,3736234,...,0,0,0,2016-01-01 02:26:05.731,yes,312.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(17.0, 0.0, 0.0, 0.0, 4.0, 3736234.0, 134217728.0, 34619480.0, 30883246.0, 0.0, 0.0, 0.0, 0.0, 0...",0.0,"(0.9473679756432686, 0.0, 0.0, 0.0, 0.3336051891155314, 1.067689114752116, 0.0, 1.63191351473157..."
2,0a50e09262,1451609770986,2.0,1.4,35,0,0,0,8,32466,...,0,0,0,2016-01-01 02:56:10.88,yes,672.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(35.0, 0.0, 0.0, 0.0, 8.0, 32466.0, 134217728.0, 31711276.0, 31678810.0, 0.0, 0.0, 0.0, 0.0, 0.0...",0.0,"(1.9504634792655529, 0.0, 0.0, 0.0, 0.6672103782310628, 0.0092776830357901, 0.0, 1.4948248752951..."
3,0a50e09262,1451608601266,2.0,0.3,13,0,0,0,3,9853730,...,0,0,0,2016-01-01 02:36:41.155,yes,439.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(13.0, 0.0, 0.0, 0.0, 3.0, 9853730.0, 134217728.0, 34230824.0, 24377094.0, 0.0, 0.0, 0.0, 0.0, 0...",0.0,"(0.7244578637272053, 0.0, 0.0, 0.0, 0.25020389183664854, 2.815862245433869, 0.0, 1.6135928184362..."
4,0a50e09262,1451606446297,2.0,0.3,11,0,0,0,2,10645694,...,0,0,0,2016-01-01 02:00:46.161,no,8.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(11.0, 0.0, 0.0, 0.0, 2.0, 10645694.0, 134217728.0, 31261076.0, 20615382.0, 0.0, 0.0, 0.0, 0.0, ...",1.0,"(0.6130028077691738, 0.0, 0.0, 0.0, 0.1668025945577657, 3.0421787293788105, 0.0, 1.4736030815440..."


In [34]:
train, test = pipeline_df.randomSplit([0.8,0.2], seed=56)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

Training Dataset Count: 7520
Test Dataset Count: 1787


In [35]:
train.groupby("rand").count().show()

+----+-----+
|rand|count|
+----+-----+
|  no| 3751|
| yes| 3769|
+----+-----+



In [36]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=5)
lrModel = lr.fit(train)
predictions = lrModel.transform(test)
#predictions_train = lrModel.transform(train)
predictions.select('label', 'features', 'rawPrediction', 'prediction', 'probability').toPandas().head()

,label,features,rawPrediction,prediction,probability
0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.8894453515307853, 0.0, 1.4704931601292144, 1.2880909544806276, 0.0, ...","[0.0079944805493118, -0.0079944805493118]",0.0,"[0.5019986094927918, 0.49800139050720815]"
1,1.0,"(0.6130028077691738, 0.0, 0.0, 0.0, 0.1668025945577657, 3.0421787293788105, 0.0, 1.4736030815440...","[0.018382919049691575, -0.018382919049691575]",0.0,"[0.5045956003465605, 0.4954043996534395]"
2,0.0,"(0.33436516787409476, 0.0, 0.0, 1.0496025671002176, 0.5838090809521799, 0.4233002181550309, 0.0,...","[0.007155678133120111, -0.007155678133120111]",0.0,"[0.5017889119000398, 0.4982110880999602]"
3,1.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 3.55954686924749, 0.0, 1.4770399562272487, 0.8658869802763666, 0.0, 0....","[0.03016442388168067, -0.03016442388168067]",0.0,"[0.5075405342228161, 0.4924594657771839]"
4,0.0,"(0.33436516787409476, 0.0, 0.0, 0.0, 0.08340129727888285, 1.8311298938973548, 0.0, 1.36423693041...","[0.01371295746760036, -0.01371295746760036]",0.0,"[0.5034281856459132, 0.4965718143540868]"


In [38]:
predictions.groupBy("prediction").count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|       0.0| 1223|
|       1.0|  564|
+----------+-----+



In [104]:
import pyspark.ml.evaluation as ev

evaluator = ev.BinaryClassificationEvaluator(rawPredictionCol='probability', labelCol='label')

print(evaluator.evaluate(predictions, {evaluator.metricName: 'areaUnderROC'}))
print(evaluator.evaluate(predictions, {evaluator.metricName: 'areaUnderPR'}))

0.5074512340988931
0.48283923439972604


In [41]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

In [64]:
gbt = GBTRegressor(featuresCol = "features", maxIter=10)

In [65]:
gbtModel = gbt.fit(train)

In [66]:
preds = gbtModel.transform(test)

In [67]:
preds.toPandas().head()

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,...,_c23,_c24,_c25,rand,_c25_index,_c25_encoded,vectorized_features,label,features,prediction
0,0a50e09262,1451606410959,2.0,2.4,0,0,0,0,0,3112494,...,0,0,2016-01-01 02:00:10.839,yes,1.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 3112494.0, 134217728.0, 31195102.0, 28082608.0, 0.0, 0.0, 0.0, 0.0, 0....",0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.8894453515307853, 0.0, 1.4704931601292144, 1.2880909544806276, 0.0, ...",0.487897
1,0a50e09262,1451606446297,2.0,0.3,11,0,0,0,2,10645694,...,0,0,2016-01-01 02:00:46.161,no,8.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(11.0, 0.0, 0.0, 0.0, 2.0, 10645694.0, 134217728.0, 31261076.0, 20615382.0, 0.0, 0.0, 0.0, 0.0, ...",1.0,"(0.6130028077691738, 0.0, 0.0, 0.0, 0.1668025945577657, 3.0421787293788105, 0.0, 1.4736030815440...",0.516752
2,0a50e09262,1451606545618,2.0,2.4,6,0,0,22,7,1481282,...,0,0,2016-01-01 02:02:25.515,yes,28.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(6.0, 0.0, 0.0, 22.0, 7.0, 1481282.0, 134217728.0, 34341020.0, 32859738.0, 0.0, 0.0, 0.0, 0.0, 0...",0.0,"(0.33436516787409476, 0.0, 0.0, 1.0496025671002176, 0.5838090809521799, 0.4233002181550309, 0.0,...",0.486184
3,0a50e09262,1451606555646,2.0,2.4,0,0,0,0,0,12456154,...,0,0,2016-01-01 02:02:35.489,no,30.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 12456154.0, 134217728.0, 31333986.0, 18877832.0, 0.0, 0.0, 0.0, 0.0, 0...",1.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 3.55954686924749, 0.0, 1.4770399562272487, 0.8658869802763666, 0.0, 0....",0.450917
4,0a50e09262,1451606575899,2.0,1.4,6,0,0,0,1,6407792,...,0,0,2016-01-01 02:02:55.772,yes,34.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(6.0, 0.0, 0.0, 0.0, 1.0, 6407792.0, 134217728.0, 28940978.0, 22533186.0, 0.0, 0.0, 0.0, 0.0, 0....",0.0,"(0.33436516787409476, 0.0, 0.0, 0.0, 0.08340129727888285, 1.8311298938973548, 0.0, 1.36423693041...",0.510510


In [84]:
preds.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: long (nullable = true)
 |-- _c2: double (nullable = true)
 |-- _c3: double (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: integer (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: integer (nullable = true)
 |-- _c14: integer (nullable = true)
 |-- _c15: integer (nullable = true)
 |-- _c16: integer (nullable = true)
 |-- _c17: integer (nullable = true)
 |-- _c18: integer (nullable = true)
 |-- _c19: integer (nullable = true)
 |-- _c20: integer (nullable = true)
 |-- _c21: integer (nullable = true)
 |-- _c22: integer (nullable = true)
 |-- _c23: integer (nullable = true)
 |-- _c24: integer (nullable = true)
 |-- _c25: string (nullable = true)
 |-- rand: string (nullable = false)
 |-- _c25_ind

In [80]:
predsInt = preds

In [96]:
import pyspark.sql.functions as func
BiPred = predsInt.withColumn("preds", func.round(predsInt["prediction"]).cast('integer'))

In [97]:
BiPred.toPandas().head()

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,...,_c24,_c25,rand,_c25_index,_c25_encoded,vectorized_features,label,features,prediction,preds
0,0a50e09262,1451606410959,2.0,2.4,0,0,0,0,0,3112494,...,0,2016-01-01 02:00:10.839,yes,1.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 3112494.0, 134217728.0, 31195102.0, 28082608.0, 0.0, 0.0, 0.0, 0.0, 0....",0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.8894453515307853, 0.0, 1.4704931601292144, 1.2880909544806276, 0.0, ...",0.487897,0
1,0a50e09262,1451606446297,2.0,0.3,11,0,0,0,2,10645694,...,0,2016-01-01 02:00:46.161,no,8.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(11.0, 0.0, 0.0, 0.0, 2.0, 10645694.0, 134217728.0, 31261076.0, 20615382.0, 0.0, 0.0, 0.0, 0.0, ...",1.0,"(0.6130028077691738, 0.0, 0.0, 0.0, 0.1668025945577657, 3.0421787293788105, 0.0, 1.4736030815440...",0.516752,1
2,0a50e09262,1451606545618,2.0,2.4,6,0,0,22,7,1481282,...,0,2016-01-01 02:02:25.515,yes,28.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(6.0, 0.0, 0.0, 22.0, 7.0, 1481282.0, 134217728.0, 34341020.0, 32859738.0, 0.0, 0.0, 0.0, 0.0, 0...",0.0,"(0.33436516787409476, 0.0, 0.0, 1.0496025671002176, 0.5838090809521799, 0.4233002181550309, 0.0,...",0.486184,0
3,0a50e09262,1451606555646,2.0,2.4,0,0,0,0,0,12456154,...,0,2016-01-01 02:02:35.489,no,30.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 12456154.0, 134217728.0, 31333986.0, 18877832.0, 0.0, 0.0, 0.0, 0.0, 0...",1.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 3.55954686924749, 0.0, 1.4770399562272487, 0.8658869802763666, 0.0, 0....",0.450917,0
4,0a50e09262,1451606575899,2.0,1.4,6,0,0,0,1,6407792,...,0,2016-01-01 02:02:55.772,yes,34.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(6.0, 0.0, 0.0, 0.0, 1.0, 6407792.0, 134217728.0, 28940978.0, 22533186.0, 0.0, 0.0, 0.0, 0.0, 0....",0.0,"(0.33436516787409476, 0.0, 0.0, 0.0, 0.08340129727888285, 1.8311298938973548, 0.0, 1.36423693041...",0.510510,1


In [98]:
BiPred.groupBy("preds").count().show()

+-----+-----+
|preds|count|
+-----+-----+
|    1|  779|
|    0| 1008|
+-----+-----+



In [102]:
import pyspark.ml.evaluation as ev

evals = ev.BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label')

print(evals.evaluate(BiPred, {evals.metricName: 'areaUnderROC'}))
print(evals.evaluate(BiPred, {evals.metricName: 'areaUnderPR'}))

0.4862359670305146
0.47594404287530084


In [109]:
evalu = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evalu.evaluate(preds)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)


Root Mean Squared Error (RMSE) on test data = 0.508255
